In [41]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings(action='ignore')
from matplotlib import rc, font_manager
if os.name == 'posix': 
    plt.rc('font', family='AppleGothic')
elif os.name == 'nt': 
    plt.rc('font', family='Malgun Gothic')

plt.rc('axes', unicode_minus=False) 

%config InlineBackend.figure_format = 'retina'

In [42]:
regular=pd.read_csv("Regular_Season_Batter.csv")
del regular['team']
del regular['career']

In [43]:
agg = {}
for i in regular.columns:
    agg[i] = []
for i in regular.loc[regular['year']==2018, 'batter_name'].unique():
    for j in regular.columns:
        if j in ['batter_id', 'batter_name', 'height/weight', 'year_born', 'position', 'starting_salary']:
            agg[j].append(regular.loc[regular['batter_name']==i,j].iloc[0])
        elif j in ['avg', 'G', 'AB', 'R', 'H','2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP','SLG', 'OBP', 'E','OPS']:
            agg[j].append(0)
        elif j == 'year':
            agg[j].append(2019)

In [44]:
pd.DataFrame(agg,columns=regular.columns)

,batter_id,batter_name,year,avg,G,AB,R,H,2B,3B,...,SO,GDP,SLG,OBP,E,height/weight,year_born,position,starting_salary,OPS
0,0,가르시아,2019,0,0,0,0,0,0,0,...,0,0,0,0,0,177cm/93kg,1985년 04월 12일,내야수(우투우타),NaN,0
1,1,강경학,2019,0,0,0,0,0,0,0,...,0,0,0,0,0,180cm/72kg,1992년 08월 11일,내야수(우투좌타),10000만원,0
2,2,강구성,2019,0,0,0,0,0,0,0,...,0,0,0,0,0,180cm/82kg,1993년 06월 09일,외야수(우투좌타),9000만원,0
3,3,강민국,2019,0,0,0,0,0,0,0,...,0,0,0,0,0,176cm/80kg,1992년 01월 10일,내야수(우투우타),20000만원,0
4,4,강민호,2019,0,0,0,0,0,0,0,...,0,0,0,0,0,185cm/100kg,1985년 08월 18일,포수(우투우타),9000만원,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,337,홍재호,2019,0,0,0,0,0,0,0,...,0,0,0,0,0,180cm/79kg,1987년 05월 10일,내야수(우투우타),4000만원,0
223,338,홍창기,2019,0,0,0,0,0,0,0,...,0,0,0,0,0,189cm/94kg,1993년 11월 21일,외야수(우투좌타),8000만원,0
224,341,황윤호,2019,0,0,0,0,0,0,0,...,0,0,0,0,0,177cm/78kg,1993년 09월 22일,내야수(우투우타),3000만원,0
225,342,황재균,2019,0,0,0,0,0,0,0,...,0,0,0,0,0,183cm/96kg,1987년 07월 28일,내야수(우투우타),6000만원,0


In [45]:
for j in ['avg', 'G', 'AB', 'R', 'H','2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP','SLG', 'OBP', 'E']:
    lag_1_avg=[]
    for i in range(len(regular)):
        if len(regular.loc[(regular['batter_name'] == regular['batter_name'].iloc[i])&(regular['year']==regular['year'].iloc[i]-1)]['avg'])==0:
            lag_1_avg.append(np.nan)
        else:
            lag_1_avg.append(regular.loc[(regular['batter_name'] == regular['batter_name'].iloc[i])&(regular['year']==regular['year'].iloc[i]-1)]['avg'].iloc[0])
    regular['lag_1_' + j] = lag_1_avg
    print(j)

avg
G
AB
R
H
2B
3B
HR
TB
RBI
SB
CS
BB
HBP
SO
GDP
SLG
OBP
E


In [46]:
#신뢰성이 낮은 지표: 2B/AB, avg, 3B/AB => 이 변수들은 누적  지표로

In [47]:
total_2BnAB=[]
total_3BnAB=[]
total_avg=[]
for i in range(len(regular)):
    a=regular.loc[(regular['batter_name']==regular['batter_name'])&(regular['year']<regular['year'].iloc[i])]
    total_2BnAB.append(a['2B'].sum()/a['AB'].sum())
    total_3BnAB.append(a['3B'].sum()/a['AB'].sum())
    total_avg.append((a['avg']*a['AB']).sum()/a['AB'].sum())   

In [48]:
regular['total_2BnAB'] = total_2BnAB
regular['total_3BnAB'] = total_3BnAB
regular['total_avg'] = total_avg

In [49]:
regular.columns

Index(['batter_id', 'batter_name', 'year', 'avg', 'G', 'AB', 'R', 'H', '2B',
       '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP', 'SLG',
       'OBP', 'E', 'height/weight', 'year_born', 'position', 'starting_salary',
       'OPS', 'lag_1_avg', 'lag_1_G', 'lag_1_AB', 'lag_1_R', 'lag_1_H',
       'lag_1_2B', 'lag_1_3B', 'lag_1_HR', 'lag_1_TB', 'lag_1_RBI', 'lag_1_SB',
       'lag_1_CS', 'lag_1_BB', 'lag_1_HBP', 'lag_1_SO', 'lag_1_GDP',
       'lag_1_SLG', 'lag_1_OBP', 'lag_1_E', 'total_2BnAB', 'total_3BnAB',
       'total_avg'],
      dtype='object')

In [66]:
regular = regular.loc[~regular['OPS'].isnull(),]

In [67]:
from sklearn.ensemble import RandomForestRegressor

In [68]:
#training_data
y = regular['OPS']
X = regular[[x for x in regular.columns if ('lag_' in x)|('total_' in x)]]
a = X.fillna(-1)

In [69]:
rf=RandomForestRegressor(n_estimators=500)

In [71]:
rf.fit(X.fillna(-1),y)

RandomForestRegressor(n_estimators=500)

In [ ]:
train = regular.loc[regular['year']<=2017,]
test = regular.loc[regular['year']==2018,]
y_train = train['OPS']
X_train = train[[x for x in regular.columns if ('lag_' in x)|('total_' in x)]]

y_test = test['OPS']
X_test = test[[x for x in regular.columns if ('lag_' in x)|('total_' in x)]]

In [ ]:
rf=RandomForestRegressor(n_estimators=500)
rf.fit(X_train.fillna(-1),y_train)

In [ ]:
#2017년도 값을 바탕으로 2018년도를 예측하는 predict값 wrmse
pred = rf.predict(X_test.fillna(-1))

In [ ]:
real = test['OPS']
ab = test['AB']

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(real,pred,sample_weight=ab)**0.5

In [189]:
# 규정타석 설명 => 가중치를 다르게 준다 (타석 수 마다)

In [ ]:
rf=RandomForestRegressor(n_estimators=500)
rf.fit(X_train.fillna(-1),y_train,sample_weight = train['AB'])

In [ ]:
#weight를 줬을때 wrmse => 더 낮다 
pred = rf.predict(X_test.fillna(-1))

real = test['OPS']
ab = test['AB']

from sklearn.metrics import mean_squared_error
mean_squared_error(real,pred,sample_weight=ab)**0.5

In [122]:
#####################################################이제 2019년 OPS예측#####################